In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df['target'].value_counts()

In [ ]:
import re

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download(['punkt', 'stopwords'])


def preprocess(text):
    # O(1) optmization
    stopwords_set = set(stopwords.words('english'))
    return ' '.join(
        token.lower() for token in word_tokenize(text)
        if token.isalpha() and token not in stopwords_set
    )

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=500
)

X = vectorizer.fit_transform(train_df['question_text'].apply(preprocess))

In [ ]:
y = train_df['target'].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

In [ ]:
X_, y_ = smote.fit_resample(X, y)

In [ ]:
pd.Series(y_).value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.33, random_state=42)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_val_pred = model.predict(X_val)

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_val_pred, y_val))

In [ ]:
sample_submission_df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')

In [ ]:
sample_submission_df.head()

In [ ]:
X_test = vectorizer.transform(test_df['question_text'].apply(preprocess))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
submission_df = test_df.loc[:, ('qid',)]
submission_df['prediction'] = pd.Series(y_pred)

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)